In [1]:
import networkx as nx
import csv
import math
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
G_all_roads = nx.DiGraph()
both_false=[]
with open('Straßengraph/edges_th_with_frc.csv', newline='') as csvfile:
    with open('Straßengraph/coordinates_th.csv', newline='') as coordinates:    
        spamreader = csv.reader(csvfile, delimiter=';')
        cooreader = csv.reader(coordinates, delimiter=';')
        count=0
        coorrows = list(cooreader)
        for row in spamreader:        
            Id,u,v,length,maxspeed, maxspeedtrucks, IsF, IsB, Frc=row
            #print(type(Id))
            #print(Id)
            #interestingrows=[row for idx, row in enumerate(cooreader) if idx ==int(Id)]
            #interestingrows=[row2 for idx, row2 in enumerate(cooreader) if idx ==2]
            #print(coorrow[int(Id)])
            coorrow=coorrows[int(Id)]
            #length_of_coorrow=len(coorrow[int(Id)])
            #print(interestingrows)
            if(IsF=='True'):#forward=True
            #print(length1)
                G_all_roads.add_node(u,y=coorrow[1],x=coorrow[2])
                G_all_roads.add_node(v,y=coorrow[-2],x=coorrow[-1])
                G_all_roads.add_edge(u,v,Id=Id,length=int(length),maxspeed=float(maxspeed),maxspeedtrucks=float(maxspeedtrucks),
                            y1=coorrow[1],x1=coorrow[2],y2=coorrow[-2],x2=coorrow[-1],frc=int(Frc),time=int(length)/float(maxspeedtrucks))
            #WEprint(int(list(row)[1]), int(list(row)[2]))
            if(IsB=='True'):#forward=True
                G_all_roads.add_node(u,y=coorrow[1],x=coorrow[2])
                G_all_roads.add_node(v,y=coorrow[-2],x=coorrow[-1])
                G_all_roads.add_edge(v,u,Id=Id,length=int(length),maxspeed=float(maxspeed),maxspeedtrucks=float(maxspeedtrucks),
                           y1=coorrow[1],x1=coorrow[2],y2=coorrow[-2],x2=coorrow[-1],frc=int(Frc),time=int(length)/float(maxspeedtrucks))
            #if(IsF!='True' and IsB !="True"):
            #   both_false.append(Id)
            #count+=1
            #if count>3:
            #    break

In [3]:
coory=nx.get_node_attributes(G_all_roads,"y")
coorx=nx.get_node_attributes(G_all_roads,"x")
def dist(a, b):
    x1=float(coorx[a])
    x2=float(coorx[b])
    y1=float(coory[a])
    y2=float(coory[b])
    return(math.sqrt(((x1-x2)*71)**2+((y1-y2)*111)**2)*1000*100)

## Heuristic search

first check for some nodes the difference bettween the euclidean distance and the distance from the edge attributes

In [4]:
nodelist=list(G_all_roads.nodes())

In [5]:
random.seed(42)
sample_nodes=random.sample(nodelist,20)

In [6]:
#start_node=sample_nodes[0]
#for i in range(1,len(sample_nodes)):
#    start=time.time()
#    len_1=dist(start_node,sample_nodes[i])
#    len_2=nx.astar_path_length(G_all_roads, start_node,sample_nodes[i], heuristic=dist, weight='length')
#    end=time.time()
#    print(len_2/len_1,len_2/100000,end-start)

1.2072613396177674 146.97555 1.3754701614379883
1.1720939396747547 221.37405 2.1194753646850586
1.1937509400519302 154.93744 1.2374107837677002
1.2389876064219005 51.43946 0.11693263053894043
1.1955897183368311 183.13892 1.363724946975708
1.2820849345817191 68.17375 0.23101472854614258
1.2517675992824933 133.16647 0.7375600337982178
1.3472483599744982 72.98993 0.38023829460144043
1.1889134113336794 154.74121 1.354346513748169
1.214087478092779 138.7104 1.2211155891418457
1.2465131890957408 140.62042 0.8000698089599609
1.2079013207760763 136.60461 0.9880449771881104
1.182976475070832 157.68779 1.1771540641784668
1.259730238583179 143.29351 1.361076831817627
1.1710943408924723 138.01379 1.0434341430664062
1.2321989475521087 55.56902 0.13437175750732422
1.2381117820381118 102.82701 0.5802309513092041
1.17553118147935 142.06998 0.9643604755401611
1.213451224452841 208.23508 2.790527582168579


## First Impression
- Großer Zusammenhang zwischen Länge des Weges und Berechnungszeit in Astar
- There is a big dependance between the eucledean distance and the calculationtime of Astar

- Faktor liegt bei Circa 1,2 bis 1,3, Suche mehr um es genau herauszufinden
- the factor, between the eucledean length and the real distance is mostly between 1.17 and 1.3

## Create for 100 nodes the Factor between the Astar and Eucleadean difference to determine a factor

In [6]:
random.seed(5234)
sample_nodes=random.sample(nodelist,100)
start_sample=random.sample(sample_nodes,5)

In [7]:
important_list=[]
for i in range(len(start_sample)):
    for j in range(len(sample_nodes)):
        if start_sample[i]==sample_nodes[j]:
            continue
        start=time.time()
        len_1=dist(start_sample[i],sample_nodes[j])
        len_2=nx.astar_path_length(G_all_roads, start_sample[i],sample_nodes[j], heuristic=dist, weight='length')
        end=time.time()
        important_list.append([start_sample[i],sample_nodes[j],len_2/len_1,len_2,len_1,end-start])

In [9]:
len(important_list)

495

The factor over all testsed difference between nodes is 1.206

In [11]:
edgelist=list(G_all_roads.edges)

# Lineare Regression für das Schätzen der distance Matrix

In [8]:
factorlist=[]##y
lengthlist=[]##x
midpoint_2=11280000+(16000000-11280000)/2
for item in important_list:
    if(item[4]>1300000 and item[4]<midpoint_2):
    #if(item[4]>4000000 and item[4]<16000000):
        factorlist.append(item[2])
        lengthlist.append(item[4])
factorarray=np.zeros(len(factorlist))
lengtharray=np.zeros(len(lengthlist))
for i in range(len(lengthlist)):
    lengtharray[i]=lengthlist[i]
    factorarray[i]=factorlist[i]
lengtharray=lengtharray.reshape(-1, 1)

In [9]:
model_13 = LinearRegression()
model_13.fit(lengtharray, factorarray)
model_13.coef_
type(model_13.intercept_)

numpy.float64

## Use that heuristic for creating a distance Matrix

In [12]:
random.seed(523324)
sample_edges=random.sample(edgelist,20)
#start_sample=random.sample(sample_nodes,5)
distanc_matrix_classic=np.zeros((20,20))
distanc_matrix_13=np.zeros((20,20))
distanc_matrix_40=np.zeros((20,20))

In [13]:
def give_fact_of_length_13(euc_dist):
    if(euc_dist<1300000):
        return 1.37
    elif(euc_dist<midpoint_2):
        return model_13.intercept_+ euc_dist*model_13.coef_
    else:
        return 1.177

In [19]:
sample_edges

[('9956186', '6811037'),
 ('5437693', '5261371'),
 ('7629207', '9383530'),
 ('8775519', '8775562'),
 ('7462207', '9144174'),
 ('9120528', '9076722'),
 ('9423094', '9394739'),
 ('8897373', '8868558'),
 ('9118956', '9410235'),
 ('7510353', '7510354'),
 ('8510754', '6996316'),
 ('5210312', '5195970'),
 ('9188064', '9169632'),
 ('5279368', '5279369'),
 ('7261849', '7717623'),
 ('7368360', '7220033'),
 ('8773900', '7269980'),
 ('6346392', '10231793'),
 ('12465358', '12461426'),
 ('7051155', '7394251')]

In [14]:
## With Heuristic long way from 13km:
start=time.time()
for i in range(20):
    for j in range(20):
        euc_dist=dist(sample_edges[i][0],sample_edges[j][0])
        if(euc_dist<1300000):
            distanc_matrix_13[i][j]=nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[j][0],heuristic=dist,weight='length')
            
        else:
            distanc_matrix_13[i][j]=euc_dist*give_fact_of_length_13(euc_dist)
end=time.time()
time_13=end-start
print(end-start)

0.0643155574798584


for a 500 time 500 Matrix instead of a 20 times time Matrix, we need approx the (500/20)*(500/20)=25*25=625 amount of calculation time then in the current situation

that would lead, that we would need with the 13 heuristic 0.065*625 approx 40s of computation time

with the 40 heuristic, we would need 2.2*625 approx 1375s,=22min+55s

with the classic way, we would ne 295*625 approx 107675s=29,9h =29h+54min

## Arbeiten mit den Längen

In [ ]:
(nx.astar_path_length(G_all_roads, sample_edges[0][0], sample_edges[1][0], heuristic=dist, weight='time'))

In [15]:
def tdist_60(a,b):##Heuristic für Durschnittsgeschwindigkeit von 60km/h
    return dist(a,b)/(60/36)

## Erstellen von Touren mit Neighrest Neighbors Approach

Idea, always choose the nearest neighbor of the current point to achieve the next point on the subtour,
if the subtour is longer then a specific length (or time), abort the subtour and go back to the depot

In [42]:
##Try the tour first with a 20*20 Distance Matrix

##Station left:
stations_left=list(range(1,20))
## Abruchkriterium mit maximal 500km per Tour
tourlist=[] #Listen von allen Subtouren, welche derzeitig bestehen
last_station=0#letzte Station, wird in der Iteration häufig benutzt werden
curr_tour=[]
while(len(stations_left)>0):#
    #print("in while")
    if(curr_tour==[]):#Beginn beim Depot
        min_Distanz=1000000000000#some big Number
        next_station=32 ##Out auf Bound, falls es nicht funktion
        for station in stations_left:##Kleinste Station herausgefunden, add it to function
            if(distanc_matrix_13[last_station][station]<min_Distanz):
                next_station=station
                min_Distanz=distanc_matrix_13[last_station][station]
        curr_tour.append([next_station])
        curr_tour.append(min_Distanz)
        last_station=next_station
        stations_left.remove(next_station)
    else:
        min_Distanz=1000000000000#some big Number
        next_station=32 ##Out auf Bound, falls es nicht funktion
        no_valid_Station=True#Check später in der Iteration, ob man Überhaupt noch eine Tour hinzufügen kann
        for station in stations_left:##Kleinste Station herausgefunden, add it to function
            this_station_is_valid=False#Check ob aktuell zu überprüfende Station gemäß des Abbruchkriterium valide ist
            #print(curr_tour)
            if(curr_tour[1]+distanc_matrix_13[last_station][station]+distanc_matrix_13[station][0]<50000000):
                this_station_is_valid=True
                no_valid_Station=False
            if(distanc_matrix_13[last_station][station]<min_Distanz and this_station_is_valid):
                next_station=station
                min_Distanz=distanc_matrix_13[last_station][station]
        if(no_valid_Station):##Es gibt keine Station mehr, die wir der Tour hinzufügen können
            curr_tour[1]+=distanc_matrix_13[last_station][0]#Distanz von Endstation zu Depot muss noch hinzugefügt werden
            tourlist.append(curr_tour)
            last_station=0
            curr_tour=[]
            
        else:#Es wird eine Station hinzugefügt
            curr_tour[0].append(next_station)
            curr_tour[1]+=min_Distanz
            last_station=next_station
            #print(next_station)
            stations_left.remove(next_station)

In [43]:
tourlist

[[[18, 17, 19, 15, 7, 5, 4, 14, 6, 8], 49551443.16280386],
 [[1, 13, 11], 44386912.81701267],
 [[3, 10, 16], 44407243.56600118],
 [[9, 12], 47832146.897616476]]

## Touren mit Nearest Neighbor mit Zeitlimit

In [29]:
time_matrix=np.zeros((20,20))

In [91]:
def search_tour_with_time_limt(distance_matrix,timelimit=9,size=20,time_matrix=np.zeros((20,20)),sample_edges=sample_edges):
#def search_tour_with_time_limt(distance_matrix,timelimit=9,size=20,time_matrix,sample_edges):
    stations_left=list(range(1,size))
## Abruchkriterium mit maximal timelimit per Tour
    tourlist=[] #Listen von allen Subtouren, welche derzeitig bestehen
    last_station=0#letzte Station, wird in der Iteration häufig benutzt werden
    curr_tour=[]
    while(len(stations_left)>0):#
    #print("in while")
        if(curr_tour==[]):#Beginn beim Depot
            #print("wieder hier")
            min_Distanz=1000000000000#some big Number
            next_station=32 ##Out auf Bound, falls es nicht funktion
            for station in stations_left:##Kleinste Station herausgefunden, add it to function
                if(distance_matrix[last_station][station]<min_Distanz):
                    next_station=station
                    min_Distanz=distance_matrix[last_station][station]
            #print(next_station)
            ##Station ist klar, jetzt kann die Zeit zur nächsten Station berechnet werden
            add_time=0
            if(time_matrix[last_station][next_station]>0):
                add_time=time_matrix[last_station][next_station]
            else:
                time_matrix[last_station][next_station]=nx.astar_path_length(G_all_roads,sample_edges[last_station][0],sample_edges[next_station][0],heuristic=tdist_60,weight='time')/(3600*1000)
                add_time=time_matrix[last_station][next_station]
            curr_tour.append([next_station])
            curr_tour.append(min_Distanz)
            curr_tour.append(add_time)
            last_station=next_station
            stations_left.remove(next_station)
            #print(curr_tour,stations_left)
        else:
            min_Distanz=1000000000000#some big Number
            next_station=32 ##Out auf Bound, falls es nicht funktion
            no_valid_Station=True#Check später in der Iteration, ob man Überhaupt noch eine Tour hinzufügen kann
            stations_for_this_station=stations_left.copy()
            
            counter=0
            no_more_valid_points=True
            #while((counter<5 or counter<len(stations_for_this_station)) and len(stations_for_this_station)>0):#Es reicht die ersten fünf zu überprüfen, sehr unwahrscheinlich, dass eine Strecke, mit längerer Entfernung eine kürzere Zeit in Anspruch nimmt
            while(counter<5 and len(stations_for_this_station)>counter):
            #while(counter==0):
                min_Distanz=1000000000000
                for station in stations_for_this_station:##Kleinste Station herausgefunden, add it to function
                    #this_station_is_valid=False#Check ob aktuell zu überprüfende Station gemäß des Abbruchkriterium valide ist
            #print(curr_tour)
                    if(distance_matrix[last_station][station]<min_Distanz):
                        next_station=station
                        min_Distanz=distance_matrix[last_station][station]
                add_time=0
                if(time_matrix[last_station][next_station]>0):
                    add_time=time_matrix[last_station][next_station]
                else:
                    time_matrix[last_station][next_station]=nx.astar_path_length(G_all_roads,sample_edges[last_station][0],sample_edges[next_station][0],heuristic=tdist_60,weight='time')/(3600*1000)
                    add_time=time_matrix[last_station][next_station]
                back_to_depot_time=0
                if(time_matrix[next_station][0]>0):
                    back_to_depot_time=time_matrix[next_station][0]
                else:
                    time_matrix[next_station][0]=nx.astar_path_length(G_all_roads,sample_edges[next_station][0],sample_edges[0][0],heuristic=tdist_60,weight='time')/(3600*1000)
                    back_to_depot_time=time_matrix[next_station][0]
                if(curr_tour[2]+add_time+back_to_depot_time<timelimit):#Ausgewählte station is valide
                    ##Hinzufügen der Station zur Tour
                    curr_tour[0].append(next_station)
                    curr_tour[1]+=min_Distanz
                    curr_tour[2]+=add_time
                    #print(curr_tour)
                    stations_left.remove(next_station)
                    last_station=next_station
                    no_more_valid_points=False
                    if(len(stations_left)==0):##Tour muss hinzugefügt werden, Programm geht in oberste while Schleife, weil len==0 Bedingung nicht erfüllt-> Tour würde nicht hinzugefügt werden
                        curr_tour[1]+=distance_matrix[next_station][0]
                        curr_tour[2]+=back_to_depot_time
                        tourlist.append(curr_tour)
                    break
                else:
                    counter+=1
                    #print(next_station,stations_for_this_station,counter)
                    stations_for_this_station.remove(next_station)#im Nächsten Versuch der While Schleife soll natürlich diese Station nicht mehr abgefragt werden
            if(no_more_valid_points):#->Es wurde keine weitere Station gefunden, welche das Kriterium erfüllt, zurück zum Depot
                back_to_depot_time
                if(time_matrix[last_station][0]>0):
                    back_to_depot_time=time_matrix[last_station][0]
                else:
                    time_matrix[last_station][0]=nx.astar_path_length(G_all_roads,sample_edges[last_station][0],sample_edges[0][0],heuristic=tdist_60,weight='time')/(3600*1000)
                    back_to_depot_time=time_matrix[last_station][0]
                curr_tour[1]+=distance_matrix[last_station][0]
                curr_tour[2]+=back_to_depot_time
                tourlist.append(curr_tour)
                curr_tour=[]
                last_station=0
    return tourlist

In [92]:
search_tour_with_time_limt(distanc_matrix_13,9,20,time_matrix)

[[[18, 17, 19, 15, 7, 5, 4, 14, 6], 44761994.71224299, 8.663843873783975],
 [[8, 1, 13, 11], 45292093.475845724, 8.658722748129973],
 [[3, 10, 16, 2], 54713613.95026906, 8.165486374983836],
 [[9, 12], 47832146.897616476, 7.313923330793774]]

In [ ]:
## Measure the Runtime of this algorithm

In [96]:
time_matrix=np.zeros((20,20))

In [97]:
begin=time.time()
search_tour_with_time_limt(distanc_matrix_13,9,20,time_matrix)
end=time.time()
print(end-begin)

22.66212511062622


In [98]:
##Runtime, when time Distance Matrix is already meassured:
begin=time.time()
search_tour_with_time_limt(distanc_matrix_13,9,20,time_matrix)
end=time.time()
print(end-begin)

0.0004775524139404297


## Compare that Runtime with the Runtime, from the other algorithm

### Erstelle ein Liste von Subtouren und hänge die entsprechenden Zeiten an, die pro Tour benötigt werden

In [ ]:
##You have to measure the time from the Starttours in the beginn too, so add this

In [100]:
begin=time.time()
tourlist_with_times=[]
for i in range(1,20):##Es soll jeweils eine Liste mit den entsprechenden Zeiten für die Tour erstellt werden
    ##Die Liste besteht aus einer Liste mit Anfahrtsknoten, und einer Liste von Zeiten, die Liste von Zeiten
    ##soll aus drei Elementen bestehen, der 
    times=[nx.astar_path_length(G_all_roads,sample_edges[0][0],sample_edges[i][0],heuristic=tdist_60,weight='time')/(3600*1000),0,
          nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=tdist_60,weight='time')/(3600*1000)]
    #tour_with_time=[[i],times,distanc_matrix_13[0][i]/100000]
    tour_with_time=[[i],times]
    tourlist_with_times.append(tour_with_time)
end=time.time()
print(end-begin)
#print(tourlist_with_times)

32.947712421417236


In [101]:
##Combine tour i and j in tour to tour [i,j]
def combine_with_time(i,j):
    last_stop_tour_i=tourlist_with_times[i][0][-1]
    first_stop_tour_j=tourlist_with_times[j][0][0]
    tourlist_with_times[i][0]+=tourlist_with_times[j][0]
    tourlist_with_times[i][1][1]+=nx.astar_path_length(G_all_roads, sample_edges[last_stop_tour_i][0], sample_edges[first_stop_tour_j][0], heuristic=tdist_60, weight='time')/(3600*1000)
    tourlist_with_times[i][1][1]+=tourlist_with_times[j][1][1]
    tourlist_with_times[i][1][2]=tourlist_with_times[j][1][2]
    tourlist_with_times.remove(tourlist_with_times[j])

In [102]:
def length_of_all_tours(tourlist,distance_matrix):
    length=0
    for tour in tourlist:
        length_tour=0
        length_tour+=distance_matrix[0][tour[0][0]]
        for i in range(1,len(tour[0])):#Len of tour[0] because in tour[0] is the list with all the stations of the subtour
            #print(len(tour))
            #print(i)
            length_tour+=distance_matrix[tour[0][i-1]][tour[0][i]]
        length_tour+=distance_matrix[tour[0][-1]][0]
        length+=length_tour
    return length

In [104]:
def search_short_tour_timelimit_depot_to_depot(tourlist,distance_matrix,timelimit):
    #start_value=length_of_all_tours(tourlist,distance_matrix)
    maxdiff=0
    combinetour_1=-1
    combinetour_2=-1
    for i in range(0,len(tourlist)):    
        for j in range(0,i):
            curr_diff=distance_matrix[tourlist[i][0][-1]][0]+distance_matrix[0][tourlist[j][0][0]]-distance_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]
        #Der Einzusparende Weg beschreibt sich durch Endpunkt i zu Depot, Depot zu Begin j - Endpunkt i Startpunkt j
            if(curr_diff>maxdiff):#Wenn der gesparte Weg maximal ist, save Change Option 1 und 2
                #Überprüfe erst hier auf das Zeitlimit, da es nur im Fall curr_diff>maxdiff benötigt wird
                #und einiges an Rechnenzeit benötigt
                #Überprüfe, ob dies Schonmal berechnet wurde
                combine_time=0
                if(time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]>0):
                    combine_time=time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]
                else:
                    combine_time=nx.astar_path_length(G_all_roads, sample_edges[tourlist[i][0][-1]][0], sample_edges[tourlist[j][0][0]][0], heuristic=tdist_60, weight='time')/(3600*1000)
                    time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]=combine_time
                if(combine_time+tourlist[i][1][1]+tourlist[j][1][1]+tourlist[i][1][0]+tourlist[j][1][2]<timelimit):
                    #tourlist[i][1][0] Zeit vom Depot zum Start von Tour i
                    #tourlist[j][1][2] Zeit vom Ende der Tour j zum Depot
                    maxdiff=curr_diff
                    combinetour_1=i
                    combinetour_2=j
        for j in range(i+1,len(tourlist)):
            curr_diff=distance_matrix[tourlist[i][0][-1]][0]+distance_matrix[0][tourlist[j][0][0]]-distance_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]
            if(curr_diff>maxdiff):#Wenn der gesparte Weg maximal ist, save Change Option 1 und 2
                combine_time=0
                if(time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]>0):
                    combine_time=time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]
                else:
                    combine_time=nx.astar_path_length(G_all_roads, sample_edges[tourlist[i][0][-1]][0], sample_edges[tourlist[j][0][0]][0], heuristic=tdist_60, weight='time')/(3600*1000)
                    time_matrix[tourlist[i][0][-1]][tourlist[j][0][0]]=combine_time
                if(combine_time+tourlist[i][1][1]+tourlist[j][1][1]+tourlist[i][1][0]+tourlist[j][1][2]<timelimit):
                    maxdiff=curr_diff
                    combinetour_1=i
                    combinetour_2=j
    return(combinetour_1,combinetour_2)

In [106]:
time_matrix=np.zeros((20,20))

In [108]:
begin=time.time()
still_time_reserve=True
while(len(tourlist_with_times)>3 and still_time_reserve):
    i_j=search_short_tour_timelimit_depot_to_depot(tourlist_with_times,distanc_matrix_13,9)
    if(i_j==[-1,-1]):
        still_time_reserve=False
    else:
        combine_with_time(i_j[0],i_j[1])
end=time.time()
print(begin-end)

-35.69537949562073


In [109]:
tourlist_with_times

[[[4, 6, 11, 13, 1, 8],
  [2.2795813525062543, 3.800682443054232, 2.758713487230837]],
 [[9, 12, 2, 16, 3],
  [3.4812060193688974, 3.1860393447208244, 2.294164324312118]],
 [[14, 5, 7, 10, 15],
  [2.501352756404942, 4.48766806986368, 1.915878668536517]]]

In [ ]:
##35+35 seconds adds up 70s vs 20s in the other case
##here you have to check every combination of two subtours, in the other one you just go throw one element at each

In [133]:
tourlist_with_times=[]
for i in range(1,20):##Es soll jeweils eine Liste mit den entsprechenden Zeiten für die Tour erstellt werden
    ##Die Liste besteht aus einer Liste mit Anfahrtsknoten, und einer Liste von Zeiten, die Liste von Zeiten
    ##soll aus drei Elementen bestehen, der 
    times=[nx.astar_path_length(G_all_roads,sample_edges[0][0],sample_edges[i][0],heuristic=tdist_60,weight='time')/(3600*1000),0,
          nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=tdist_60,weight='time')/(3600*1000)]
    #tour_with_time=[[i],times,distanc_matrix_13[0][i]/100000]
    tour_with_time=[[i],times]
    tourlist_with_times.append(tour_with_time)

In [134]:
still_time_reserve=True
while(len(tourlist_with_times)>3 and still_time_reserve):
    i_j=search_short_tour_timelimit_depot_to_depot(tourlist_with_times,distanc_matrix_13,9)
    if(i_j==[-1,-1]):
        still_time_reserve=False
    else:
        combine_with_time(i_j[0],i_j[1])

In [135]:
tourlist_with_times

[[[1, 8, 5, 7], [3.34899294281731, 2.6585437580759828, 2.4050088283061126]],
 [[2], [4.962907552922863, 0, 4.7785322619087625]],
 [[4, 6, 11, 13, 14],
  [2.7642788094580806, 3.455875525281443, 2.7691209210833674]]]

In [136]:
time_matrix

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 4.15070669, 3.70958939, 1.75807098,
        2.32781927, 1.74169241, 2.46182065, 0.50460645, 3.18843967,
        4.37831302, 1.46303946, 3.2148899 , 1.44158234, 1.28833541,
        3.11345857, 4.06796045, 3.67390408, 3.49336503, 3.68341911],
       [0.        , 4.01125812, 0.        , 1.69935419, 2.70680892,
        2.52496766, 2.38722331, 2.62543023, 3.59184571, 1.22977073,
        1.85265387, 2.66322312, 1.28208917, 2.91197576, 2.92426189,
        2.59157041, 1.24963091, 4.68569064, 4.72100056, 3.30904488],
       [0.        , 3.91484886, 1.72770547, 0.        , 2.2112366 ,
        1.58923009, 2.25361649, 1.52328832, 3.40824148, 1.93000378,
        0.55619746, 2.36611147, 2.04554238, 2

In [142]:
begin=time.time()
print(nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],weight='time')/(3600*1000))
end=time.time()
print(end-begin)

begin=time.time()
print(nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=dist,weight='time')/(3600*1000))
end=time.time()
print(end-begin)
#Gibt eindeutig mit heuristic disc anderes Ergebnis und anderes Rechenzeit aus

1.6847103533506445
1.2310457229614258
2.079828029110536
0.023407459259033203


In [143]:
begin=time.time()
nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=dist,weight='length')/(3600*1000)
end=time.time()
print(end-begin)

0.3379523754119873


In [ ]:
##Alternative heuristic für Astar
##gehe von einer Durschnittgeschwindigkeit von 50km/h aus /=3.6 m/s /10 cm/ms

In [148]:
50/36 #50 km/h sind .... cm/ms

1.3888888888888888

In [ ]:
#t=length/spped

In [149]:
##Zeitheurist depend on dist
def tdist(a,b):
    return dist(a,b)/1.3888888888

In [150]:
begin=time.time()
print(nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=tdist,weight='time')/(3600*1000))
end=time.time()
print(end-begin)

1.6851067352513482
0.07683610916137695


### Testen dieser Heuristic

In [155]:
random.seed(42)
test_samples_1=random.sample(nodelist,50)
test_samples_2=random.sample(nodelist,50)
complete_1=np.zeros(50)
complete_2=np.zeros(50)
compare=np.zeros(50)
for i in range(50):
    complete_1[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],heuristic=tdist,weight='time')/(3600*1000)
    complete_2[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],weight='time')/(3600*1000)
    compare[i]=complete_1[i]/complete_2[i]
    print(complete_1[i],complete_2[i],compare[i])

0.9490989382416681 0.9436876800973975 1.0057341621157037
2.223068445884355 2.2043917916559055 1.0084724749471234
1.3291678864599106 1.3243601001065635 1.0036302712177452
1.6360941916995206 1.6360941916995206 1.0
0.683772581913019 0.683772581913019 1.0
1.5450730263523835 1.533207277268209 1.0077391682521337
3.2097844594007006 3.1666859028795917 1.013609987805206
4.157507947592512 4.157507947592512 1.0
0.47954670585234377 0.47954670585234377 1.0
1.7168920810643966 1.7168920810643966 1.0
2.582436206334995 2.582436206334995 1.0
3.9785202943624465 3.925480177366893 1.0135117526007051
2.03658136275994 2.03658136275994 1.0
0.6134470546327615 0.5920288118390397 1.0361777034587043
1.568561401524167 1.568561401524167 1.0
2.6698197340422465 2.6698197340422465 1.0
0.8593326447051705 0.8593326447051705 1.0
1.823898559436906 1.823898559436906 1.0
1.5253607599558376 1.5253607599558376 1.0
2.5340081074719945 2.4912010773728355 1.0171832898146875
1.3447373368925142 1.2201485972867023 1.1021094806672445

In [157]:
print(max(compare))
print(np.mean(compare))
#Man erhällt durch die Verwendung von einen Falschen Astar nur eine obere Zeitschranke der Benötigten Zeit von a nach b
#Weniger Problematisch als kleinerer Wert, da somit in der Erstellung der Touren gewährleistet werden kann, dass
# nicht mehr als die gesetzlich vorgeschriebene Lenkzeit gefahren wird.

#versuche es nochmal mit ungenauerer Abschätzung:

1.1357797948494655
1.0221041519114444


In [158]:
def tdist_60(a,b):##Heuristic für Durschnittsgeschwindigkeit von 60km/h
    return dist(a,b)/(60/36)

In [159]:
complete_1=np.zeros(50)
complete_2=np.zeros(50)
compare=np.zeros(50)
for i in range(50):
    complete_1[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],heuristic=tdist_60,weight='time')/(3600*1000)
    complete_2[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],weight='time')/(3600*1000)
    compare[i]=complete_1[i]/complete_2[i]
    print(complete_1[i],complete_2[i],compare[i])

0.9436876800973975 0.9436876800973975 1.0
2.2043917916559055 2.2043917916559055 1.0
1.3243601001065635 1.3243601001065635 1.0
1.6360941916995206 1.6360941916995206 1.0
0.683772581913019 0.683772581913019 1.0
1.533207277268209 1.533207277268209 1.0
3.1666859028795917 3.1666859028795917 1.0
4.157507947592512 4.157507947592512 1.0
0.47954670585234377 0.47954670585234377 1.0
1.7168920810643966 1.7168920810643966 1.0
2.582436206334995 2.582436206334995 1.0
3.925480177366893 3.925480177366893 1.0
2.03658136275994 2.03658136275994 1.0
0.5920288118390397 0.5920288118390397 1.0
1.568561401524167 1.568561401524167 1.0
2.6698197340422465 2.6698197340422465 1.0
0.8593326447051705 0.8593326447051705 1.0
1.823898559436906 1.823898559436906 1.0
1.5253607599558376 1.5253607599558376 1.0
2.4912010773728355 2.4912010773728355 1.0
1.2201485972867023 1.2201485972867023 1.0
0.6730092306079406 0.6730092306079406 1.0
3.2381752563401243 3.2381752563401243 1.0
1.9979246883311865 1.9979246883311865 1.0
1.729897

In [163]:
print(max(compare))
print(np.mean(compare))

1.020613126142483
1.0009151281165167


In [164]:
## Compare Performance of Astar with 50 and 60% estimated average Speed

In [171]:
speedup=np.zeros(50)
for i in range(50):
    begin_50=time.time()
    complete_1[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],heuristic=tdist,weight='time')/(3600*1000)
    end_50=time.time()
    
    begin_60=time.time()
    complete_2[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],heuristic=tdist_60,weight='time')/(3600*1000)
    end_60=time.time()
    print("50 vs 60",end_50-begin_50,end_60-begin_60)
    print("Speedup 50 to 60",(end_60-begin_60)/(end_50-begin_50))
    speedup[i]=(end_60-begin_60)/(end_50-begin_50)

50 vs 60 0.13439083099365234 0.13941478729248047
Speedup 50 to 60 1.0373831775700935
50 vs 60 0.07457447052001953 0.14487862586975098
Speedup 50 to 60 1.9427375730526746
50 vs 60 0.1666240692138672 0.4331624507904053
Speedup 50 to 60 2.5996391327739556
50 vs 60 0.42340517044067383 0.626056432723999
Speedup 50 to 60 1.4786225498200902
50 vs 60 0.04890871047973633 0.0745091438293457
Speedup 50 to 60 1.52343300607396
50 vs 60 0.06618261337280273 0.19106841087341309
Speedup 50 to 60 2.88698800389063
50 vs 60 0.22218966484069824 0.2975757122039795
Speedup 50 to 60 1.3392869214566314
50 vs 60 0.557154655456543 3.554900884628296
Speedup 50 to 60 6.380456215905337
50 vs 60 0.01320505142211914 0.015743732452392578
Speedup 50 to 60 1.1922507492868233
50 vs 60 0.34033632278442383 0.7022655010223389
Speedup 50 to 60 2.0634456389398337
50 vs 60 0.8676629066467285 1.2598683834075928
Speedup 50 to 60 1.4520251744731458
50 vs 60 0.847895622253418 2.846219062805176
Speedup 50 to 60 3.3568035830175584
5

In [174]:
print(np.mean(speedup))
print(max(speedup))

2.2285698108775716
6.380456215905337


In [175]:
##Vergleich Sppeedup 60er zu Disjkstra
speedup=np.zeros(50)
for i in range(50):
    begin_50=time.time()
    complete_1[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],heuristic=tdist_60,weight='time')/(3600*1000)
    end_50=time.time()
    
    begin_60=time.time()
    complete_2[i]=nx.astar_path_length(G_all_roads,test_samples_1[i],test_samples_2[i],weight='time')/(3600*1000)
    end_60=time.time()
    print("50 vs 60",end_50-begin_50,end_60-begin_60)
    print("Speedup 50 to 60",(end_60-begin_60)/(end_50-begin_50))
    speedup[i]=(end_60-begin_60)/(end_50-begin_50)


50 vs 60 0.1525895595550537 0.46671509742736816
Speedup 50 to 60 3.058630608727717
50 vs 60 0.14644527435302734 3.6768336296081543
Speedup 50 to 60 25.107219374963368
50 vs 60 0.4215996265411377 1.1737127304077148
Speedup 50 to 60 2.7839510675970427
50 vs 60 0.6318883895874023 3.0764336585998535
Speedup 50 to 60 4.868634571065059
50 vs 60 0.10657882690429688 0.3438904285430908
Speedup 50 to 60 3.2266298901177564
50 vs 60 0.17627429962158203 1.8027021884918213
Speedup 50 to 60 10.226687567965289
50 vs 60 0.457059383392334 5.851772308349609
Speedup 50 to 60 12.80308975371483
50 vs 60 4.060821294784546 6.71739935874939
Speedup 50 to 60 1.6541972352678458
50 vs 60 0.022149085998535156 0.05127716064453125
Speedup 50 to 60 2.315091496232508
50 vs 60 0.7055408954620361 2.4024267196655273
Speedup 50 to 60 3.4050849994914256
50 vs 60 1.5021889209747314 5.356801509857178
Speedup 50 to 60 3.5659972158370654
50 vs 60 2.8316478729248047 5.8687427043914795
Speedup 50 to 60 2.072553851241985
50 vs 60

In [176]:
print(np.mean(speedup))
print(max(speedup))
#60er Approximation deutlich schneller als dijkstra, von daher durchaus sinnvoll diese zu nehmen

7.197583978416431
40.1833417265581


In [184]:
## Vergleichen der Resultierenden Touren 13er Distanz Heuristic und "Dijkstra" Berechneter Matrix
tourlist_with_times

[[[1], [3.001430180086722, 0, 2.991308751797267]],
 [[2], [3.6020001221959315, 0, 3.608852318498066]],
 [[3], [2.300070679953922, 0, 2.294164324312118]],
 [[4], [2.2795813525062543, 0, 2.2472817898522433]],
 [[5], [2.14960433945284, 0, 2.1199945575647106]],
 [[6], [2.711976302131432, 0, 2.6811231402562132]],
 [[7], [2.2458832879741455, 0, 2.248175546040066]],
 [[8], [2.737061553034224, 0, 2.758713487230837]],
 [[9], [3.4812060193688974, 0, 3.5023056556299315]],
 [[10], [2.750622905073625, 0, 2.720370651567102]],
 [[11], [3.5909450094705506, 0, 3.5737798005590706]],
 [[12], [3.5776207401090643, 0, 3.5702390306378087]],
 [[13], [3.821998952139944, 0, 3.8260598717448095]],
 [[14], [2.501352756404942, 0, 2.470241562299617]],
 [[15], [1.8259363871063996, 0, 1.915878668536517]],
 [[16], [2.8336647938771935, 0, 2.8371025735931914]],
 [[17], [0.8174281960989725, 0, 0.8154576141726012]],
 [[18], [0.30206924164520116, 0, 0.3201836449681798]],
 [[19], [1.682128309438968, 0, 1.6847103533506445]]]

In [188]:
still_time_reserve=True
while(len(tourlist_with_times)>3 and still_time_reserve):
    i_j=search_short_tour_timelimit_depot_to_depot(tourlist_with_times,distanc_matrix_13,9)
    if(i_j==[-1,-1]):
        still_time_reserve=False
    else:
        combine_with_time(i_j[0],i_j[1])

In [189]:
tourlist_with_times

[[[4, 6, 11, 13, 1, 8],
  [2.2795813525062543, 3.800682443054232, 2.758713487230837]],
 [[9, 12, 2, 16, 3],
  [3.4812060193688974, 3.1860393447208244, 2.294164324312118]],
 [[14, 5, 7, 10, 15],
  [2.501352756404942, 4.48766806986368, 1.915878668536517]]]

In [190]:
time_matrix

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 3.49634039, 3.22153691, 1.72454996,
        0.        , 1.72550499, 0.        , 0.50460645, 0.        ,
        0.        , 0.        , 0.        , 1.43661318, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.51910537, 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.14053765,
        1.60883418, 0.        , 1.19820575, 0.        , 0.        ,
        0.        , 1.07570228, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.90955679,
        1.46932633, 0.        , 0.        , 0.        , 1.81564   ,
        0.50703065, 0.        , 0.        , 0

In [191]:
tourlist_with_times=[]
for i in range(1,20):##Es soll jeweils eine Liste mit den entsprechenden Zeiten für die Tour erstellt werden
    ##Die Liste besteht aus einer Liste mit Anfahrtsknoten, und einer Liste von Zeiten, die Liste von Zeiten
    ##soll aus drei Elementen bestehen, der 
    times=[nx.astar_path_length(G_all_roads,sample_edges[0][0],sample_edges[i][0],heuristic=tdist_60,weight='time')/(3600*1000),0,
          nx.astar_path_length(G_all_roads,sample_edges[i][0],sample_edges[0][0],heuristic=tdist_60,weight='time')/(3600*1000)]
    #tour_with_time=[[i],times,distanc_matrix_13[0][i]/100000]
    tour_with_time=[[i],times]
    tourlist_with_times.append(tour_with_time)
print(tourlist_with_times)

[[[1], [3.001430180086722, 0, 2.991308751797267]], [[2], [3.6020001221959315, 0, 3.608852318498066]], [[3], [2.300070679953922, 0, 2.294164324312118]], [[4], [2.2795813525062543, 0, 2.2472817898522433]], [[5], [2.14960433945284, 0, 2.1199945575647106]], [[6], [2.711976302131432, 0, 2.6811231402562132]], [[7], [2.2458832879741455, 0, 2.248175546040066]], [[8], [2.737061553034224, 0, 2.758713487230837]], [[9], [3.4812060193688974, 0, 3.5023056556299315]], [[10], [2.750622905073625, 0, 2.720370651567102]], [[11], [3.5909450094705506, 0, 3.5737798005590706]], [[12], [3.5776207401090643, 0, 3.5702390306378087]], [[13], [3.821998952139944, 0, 3.8260598717448095]], [[14], [2.501352756404942, 0, 2.470241562299617]], [[15], [1.8259363871063996, 0, 1.915878668536517]], [[16], [2.8336647938771935, 0, 2.8371025735931914]], [[17], [0.8174281960989725, 0, 0.8154576141726012]], [[18], [0.30206924164520116, 0, 0.3201836449681798]], [[19], [1.682128309438968, 0, 1.6847103533506445]]]


In [192]:
still_time_reserve=True
while(len(tourlist_with_times)>3 and still_time_reserve):
    i_j=search_short_tour_timelimit_depot_to_depot(tourlist_with_times,distanc_matrix_classic,9)
    if(i_j==[-1,-1]):
        still_time_reserve=False
    else:
        combine_with_time(i_j[0],i_j[1])

In [193]:
tourlist_with_times

[[[4, 6, 11, 13, 1, 8],
  [2.2795813525062543, 3.800682443054232, 2.758713487230837]],
 [[9, 12, 2, 16, 3, 18],
  [3.4812060193688974, 5.160020024064762, 0.3201836449681798]],
 [[15, 10, 7, 5, 14],
  [1.8259363871063996, 4.489428624982601, 2.470241562299617]]]

In [ ]:
## Man kann tatsächlich beobachten, dass mit der klassichen Distanz Matrix eine andere Tourenliste herauskommt wie
#mit der heuristic für die Distanzen 